<a href="https://colab.research.google.com/github/mahn-bonnie/Generative-AI-Series/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ML | Text Generation using Gated Recurrent Unit Networks:
Build a Text Generator by building a Gated Recurrent Unit Network.**

In [ ]:
#The conceptual procedure of training the network is to first feed the network a mapping of each character present in the text
#on which the network is training to a unique number.

**Step 1: Importing the required libraries**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

from keras.optimizers import RMSprop

from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import random
import sys


**Step 2: Loading the data into a string**

In [ ]:

# Reading the text file into a string
with open('poems.txt', 'r') as file:
	text = file.read()

# A preview of the text file
print(text)


﻿Buffalo Bill's 
defunct 
who used to 
ride a watersmooth-silver 
stallion 
and break one two three four five pigeons just like that 
Jesus 


he was a handsome man 
and what i want to know is 
how do you like your blueeyed boy 
Mister Death 
Had I the heaven's embroidered cloths, 
Enwrought with golden and silver light, 
The blue and the dim and the dark cloths 
Of night and light and the half-light, 
I would spread the cloths under your feet: 
But I, being poor, have only my dreams; 
I have spread my dreams under your feet; 
Tread softly because you tread on my dreams. 


He clasps the crag with crooked hands; 
Close to the sun in lonely lands, 
Ring'd with the azure world, he stands.


**Step 3: Creating a mapping from each unique character in the text to a unique number**

In [ ]:
# Storing all the unique characters present in the text
vocabulary = sorted(list(set(text)))

# Creating dictionaries to map each character to an index
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

print(vocabulary)


['\n', ' ', "'", ',', '-', '.', ':', ';', 'B', 'C', 'D', 'E', 'H', 'I', 'J', 'M', 'O', 'R', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', '\ufeff']


**Step 4: Pre-processing the data**

In [ ]:
import numpy as np

# Dividing the text into subsequences of length max_length
# So that at each time step the next max_length characters
# are fed into the network
max_length = 100
steps = 5
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, steps):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])

# Hot encoding each character into a boolean vector
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = np.bool_)
y = np.zeros((len(sentences), len(vocabulary)), dtype = np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1


**Step 5: Building the GRU network**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

# Initializing the LSTM network
model = Sequential()

# Defining the cell type
model.add(GRU(128, input_shape=(max_length, len(vocabulary))))

# Defining the densely connected Neural Network layer
model.add(Dense(len(vocabulary)))

# Defining the activation function for the cell
model.add(Activation('softmax'))

# Defining the optimizing function
optimizer = RMSprop(learning_rate=0.01)  # Use learning_rate instead of lr

# Configuring the model for training
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


**Step 6: Defining some helper functions which will be used during the training of the network**

In [ ]:
#a) Helper function to sample the next character:
# Helper function to sample an index from a probability array
def sample_index(preds, temperature = 1.0):
# temperature determines the freedom the function has when generating text

	# Converting the predictions vector into a numpy array
	preds = np.asarray(preds).astype('float64')

	# Normalizing the predictions array
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)

	# The main sampling step. Creates an array of probabilities signifying
	# the probability of each character to be the next character in the
	# generated text
	probas = np.random.multinomial(1, preds, 1)

	# Returning the character with maximum probability to be the next character
	# in the generated text
	return np.argmax(probas)


In [ ]:
#b) Helper function to generate text after each epoch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau

# Helper function to generate text after the end of each epoch
def on_epoch_end(epoch, logs):
	print()
	print('----- Generating text after Epoch: % d' % epoch)

	# Choosing a random starting index for the text generation
	start_index = random.randint(0, len(text) - max_length - 1)

	# Sampling for different values of diversity
	for diversity in [0.2, 0.5, 1.0, 1.2]:
		print('----- diversity:', diversity)

		generated = ''

		# Seed sentence
		sentence = text[start_index: start_index + max_length]

		generated += sentence
		print('----- Generating with seed: "' + sentence + '"')
		sys.stdout.write(generated)

		for i in range(400):
			# Initializing the predictions vector
			x_pred = np.zeros((1, max_length, len(vocabulary)))

			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			# Making the predictions for the next character
			preds = model.predict(x_pred, verbose = 0)[0]

			# Getting the index of the most probable next character
			next_index = sample_index(preds, diversity)

			# Getting the most probable next character using the mapping built
			next_char = indices_to_char[next_index]

			# Building the generated text
			generated += next_char
			sentence = sentence[1:] + next_char

			sys.stdout.write(next_char)
			sys.stdout.flush()
		print()

# Defining a custom callback function to
# describe the internal states of the network
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)


In [ ]:
#c) Helper function to save the model after each epoch in which loss decreases

from tensorflow.keras.callbacks import ModelCheckpoint

# c) Helper function to save the model after each epoch in which loss decreases

# Defining a helper function to save the model after each epoch
# in which the loss decreases
filepath = "weights_epoch_{epoch:02d}_loss_{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')



In [ ]:
#d) Helper function to reduce the learning rate each time the learning plateaus

# Defining a helper function to reduce the learning rate each time
# the learning plateaus
reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2,
							patience = 1, min_lr = 0.001)
callbacks = [print_callback, checkpoint, reduce_alpha]


**Step 7: Training the GRU model**

In [ ]:
import random
import numpy as np
import sys
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau


# Training the GRU model
model.fit(X, y, batch_size=128, epochs=20, callbacks=callbacks)


Streaming output truncated to the last 5000 lines.

il erth ladh  ce 
neloikery hh yue uand nu hosr atthaccore e clud 
n
lyf llo  lollh u deu c
 tha ilhlo 
iur 'thncllhar
----- diversity: 1.0
----- Generating with seed: "ms; 
I have spread my dreams under your feet; 
Tread softly because you tread on my dreams. 


He cl"
ms; 
I have spread my dreams under your feet; 
Tread softly because you tread on my dreams. 


He clavlt  hesemyccaeeef,aoo fihwa
w hBubooe; ilrth auneoke sfeyyoondwv ot d Bd 
othnnivu,y oBsu l 
ou nu

ehwa,w
 
cwfuil du htes co drtw avnphl
riikcalad hauru; 
 B
ppr
mahfsh ulllirueys lcs
'auscualyaurIdwsuh ofi,rt Mb te yokthse; wtjfid k,b-i'hius 
trole  al lknsptak

gklrcDntrhtuTi;d ihessi,zds
olp aBmrer
kaa, lauivaIasmtufd ad hthe  lu vtlh iy 
h d,ald a

shcosdsi t
h ul
p
koino.utaanH
iwl, a   k
----- diversity: 1.2
----- Generating with seed: "ms; 
I have spread my dreams under your feet; 
Tread softly because you tread on my dreams. 


He cl"
ms; 
I have spread my dr

**Step 8: Generating new and random text**

In [23]:
def generate_text(length, diversity):
	# Get random starting text
	start_index = random.randint(0, len(text) - max_length - 1)

	# Defining the generated text
	generated = ''
	sentence = text[start_index: start_index + max_length]
	generated += sentence

	# Generating new text of given length
	for i in range(length):

			# Initializing the prediction vector
			x_pred = np.zeros((1, max_length, len(vocabulary)))
			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			# Making the predictions
			preds = model.predict(x_pred, verbose = 0)[0]

			# Getting the index of the next most probable index
			next_index = sample_index(preds, diversity)

			# Getting the most probable next character using the mapping built
			next_char = indices_to_char[next_index]

			# Generating new text
			generated += next_char
			sentence = sentence[1:] + next_char
	return generated

print(generate_text(500, 0.2))


Bill's 
defunct 
who used to 
ride a watersmooth-silver 
stallion 
and break one two three four five and 
iithande ande ande ande alonde and 
he ande ande andde and 
ithalithtandwthalithaande ande  au 


ie and 
ie and 
itha 



ie alithaan
ithe ande ande ande ande ande ande alu 

ie ande ande and alithand 
ithande ande ande ande ande ande ande alond wthaandwithanthande and ali ande ande and ithanthad anitheand 
he andwthande and withaand ithande and ithasuali thandhande aur  
ithand halole ande ande ande  ande and withande anthithande ande ande  



i ike clone and ithaau 

ie  ande 
onde and
